In [1]:
# Problem 1: Part 1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
from torchvision import datasets
data_path = "../dlwpt-code/p1ch7/"
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from torchvision import transforms
to_tensor = transforms.ToTensor()
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform=transforms.ToTensor())
tensor_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False, transform=transforms.ToTensor())

In [52]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=3, padding=1, device=device)
        #self.act1 = nn.Tanh()
        #self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16,8, kernel_size=3, padding=1, device=device)
        #self.act2 = nn.Tanh()
        #self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32, device=device)
        #self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32,10, device=device) # Not sure if it should be 10 or 2
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        
        
        out = out.view(-1, 8*8*8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [53]:
import time
import datetime
start_time = time.perf_counter_ns()

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs.to(device))
            loss = loss_fn(outputs, labels.to(device))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            
        if epoch == 1 or epoch % 10 == 0:
            print("{} Epoch {}, Training loss {}".format (
                datetime.datetime.now(), epoch, loss_train / len(train_loader)
            ))
    end_time = time.perf_counter_ns()

    total_time_ms = (end_time - start_time) / 1000000

    print("Total Time: %d ms" % (total_time_ms)) # Does not work for duration of over 1 minute!

In [54]:
n_epochs = 300
train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64, shuffle=True)
model = Net().to(device=device)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = n_epochs,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

2022-03-30 12:26:10.541143 Epoch 1, Training loss 2.2380613379771144
2022-03-30 12:26:58.739082 Epoch 10, Training loss 1.4227605641955305
2022-03-30 12:27:52.708332 Epoch 20, Training loss 1.1753715974138217
2022-03-30 12:28:46.455533 Epoch 30, Training loss 1.0549339225225132
2022-03-30 12:29:40.257529 Epoch 40, Training loss 0.9806089130661372
2022-03-30 12:30:33.961440 Epoch 50, Training loss 0.9297768067368462
2022-03-30 12:31:27.036329 Epoch 60, Training loss 0.8890225231037725
2022-03-30 12:32:20.796890 Epoch 70, Training loss 0.8561424019620242
2022-03-30 12:33:14.292071 Epoch 80, Training loss 0.826676774779549
2022-03-30 12:34:08.059160 Epoch 90, Training loss 0.8021137480769316
2022-03-30 12:35:01.856404 Epoch 100, Training loss 0.7789207605449745
2022-03-30 12:35:55.391150 Epoch 110, Training loss 0.7624223160240656
2022-03-30 12:36:48.678650 Epoch 120, Training loss 0.7439663814156866
2022-03-30 12:37:41.786013 Epoch 130, Training loss 0.7260743517552495
2022-03-30 12:38:3

In [55]:
train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(tensor_cifar10_val, batch_size=64, shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs.to(device))
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels.to(device)).sum())
        print("Accuracy {}: {:.2f}".format(name, correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.80
Accuracy val: 0.63


In [56]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18354, [432, 16, 1152, 8, 16384, 32, 320, 10])

In [23]:
torch.save(model.state_dict(), "cnn_cifar10.pt")

In [24]:
# Problem 1 part B

In [64]:
import torch.nn as nn
import torch.nn.functional as F
class Net_deeper(nn.Module):
    def __init__(self, n_chans1=32):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3,n_chans1,kernel_size=3, padding=1, device=device)
        self.conv2 = nn.Conv2d(n_chans1,n_chans1 // 2, kernel_size=3, padding=1, device=device)
        self.conv3 = nn.Conv2d(n_chans1 // 2,n_chans1 // 2, kernel_size=3, padding=1, device=device)
        self.fc1 = nn.Linear(4*4*n_chans1 // 2, 32, device=device)
        self.fc2 = nn.Linear(32,10, device=device)
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = F.max_pool2d(torch.tanh(self.conv3(out)), 2)
        out = out.view(-1, 4*4*self.n_chans1 // 2)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [65]:
n_epochs = 300
train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64, shuffle=True)
model = Net_deeper(n_chans1=32).to(device=device)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = n_epochs,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

2022-03-30 13:04:39.521890 Epoch 1, Training loss 2.2985282700384975
2022-03-30 13:05:36.970011 Epoch 10, Training loss 1.4462246405499062
2022-03-30 13:06:40.326698 Epoch 20, Training loss 1.161722811210491
2022-03-30 13:07:44.722435 Epoch 30, Training loss 1.0169035414295733
2022-03-30 13:08:47.935809 Epoch 40, Training loss 0.9187496081185158
2022-03-30 13:09:50.974845 Epoch 50, Training loss 0.8472257677246543
2022-03-30 13:10:53.734033 Epoch 60, Training loss 0.7991855386501688
2022-03-30 13:11:56.475326 Epoch 70, Training loss 0.760929742508837
2022-03-30 13:12:59.304192 Epoch 80, Training loss 0.7299830194019601
2022-03-30 13:14:02.360058 Epoch 90, Training loss 0.7036529038568287
2022-03-30 13:15:03.452097 Epoch 100, Training loss 0.6826299365295474
2022-03-30 13:16:04.437418 Epoch 110, Training loss 0.662505298205044
2022-03-30 13:17:05.088766 Epoch 120, Training loss 0.6444871906962846
2022-03-30 13:18:05.640252 Epoch 130, Training loss 0.6270723457607772
2022-03-30 13:19:06.

In [66]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(16394, [864, 32, 4608, 16, 2304, 16, 8192, 32, 320, 10])

In [67]:
train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64, shuffle=False)

val_loader = torch.utils.data.DataLoader(tensor_cifar10_val, batch_size=64, shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs.to(device))
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels.to(device)).sum())
        print("Accuracy {}: {:.2f}".format(name, correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.83
Accuracy val: 0.68


In [68]:
torch.save(model.state_dict(), "cnn_cifar10_partB.pt")